In [1]:
import numpy as np

In [22]:
x = np.array([1,2,3,4,5,6,7,8,9, 1,2,3,4,5,6,7,8,9])
x = x.reshape(1,2,3,3)
print(x)

[[[[1 2 3]
   [4 5 6]
   [7 8 9]]

  [[1 2 3]
   [4 5 6]
   [7 8 9]]]]


In [23]:
field_height=3
field_width=3
padding=1
stride=1

In [24]:
p = padding
x_padded = np.pad(x, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')
print(x_padded)

[[[[0 0 0 0 0]
   [0 1 2 3 0]
   [0 4 5 6 0]
   [0 7 8 9 0]
   [0 0 0 0 0]]

  [[0 0 0 0 0]
   [0 1 2 3 0]
   [0 4 5 6 0]
   [0 7 8 9 0]
   [0 0 0 0 0]]]]


In [25]:
N, C, H, W = x.shape
assert (H + 2 * padding - field_height) % stride == 0
assert (W + 2 * padding - field_height) % stride == 0

In [26]:
out_height = (H + 2 * padding - field_height) / stride + 1
out_width = (W + 2 * padding - field_width) / stride + 1
print(out_height)
print(out_width)

3.0
3.0


In [27]:
i0 = np.arange(field_height,dtype='int32')
print(i0)

[0 1 2]


In [28]:
i0 = np.repeat(i0, field_width)
print(i0)

[0 0 0 1 1 1 2 2 2]


In [29]:
i0 = np.tile(i0, C)
print(i0)

[0 0 0 1 1 1 2 2 2 0 0 0 1 1 1 2 2 2]


In [30]:
i1 = stride * np.repeat(np.arange(out_height,dtype='int32'), out_width)
print(i1)

[0 0 0 1 1 1 2 2 2]


In [31]:
j0 = np.tile(np.arange(field_width), field_height * C)
print(j0)

[0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2]


In [32]:
j1 = stride * np.tile(np.arange(out_width,dtype='int32'), int(out_height))
print(j1)

[0 1 2 0 1 2 0 1 2]


In [33]:
i = i0.reshape(-1, 1) + i1.reshape(1, -1)
print(i)

[[0 0 0 1 1 1 2 2 2]
 [0 0 0 1 1 1 2 2 2]
 [0 0 0 1 1 1 2 2 2]
 [1 1 1 2 2 2 3 3 3]
 [1 1 1 2 2 2 3 3 3]
 [1 1 1 2 2 2 3 3 3]
 [2 2 2 3 3 3 4 4 4]
 [2 2 2 3 3 3 4 4 4]
 [2 2 2 3 3 3 4 4 4]
 [0 0 0 1 1 1 2 2 2]
 [0 0 0 1 1 1 2 2 2]
 [0 0 0 1 1 1 2 2 2]
 [1 1 1 2 2 2 3 3 3]
 [1 1 1 2 2 2 3 3 3]
 [1 1 1 2 2 2 3 3 3]
 [2 2 2 3 3 3 4 4 4]
 [2 2 2 3 3 3 4 4 4]
 [2 2 2 3 3 3 4 4 4]]


In [34]:
j = j0.reshape(-1, 1) + j1.reshape(1, -1)
print(j)

[[0 1 2 0 1 2 0 1 2]
 [1 2 3 1 2 3 1 2 3]
 [2 3 4 2 3 4 2 3 4]
 [0 1 2 0 1 2 0 1 2]
 [1 2 3 1 2 3 1 2 3]
 [2 3 4 2 3 4 2 3 4]
 [0 1 2 0 1 2 0 1 2]
 [1 2 3 1 2 3 1 2 3]
 [2 3 4 2 3 4 2 3 4]
 [0 1 2 0 1 2 0 1 2]
 [1 2 3 1 2 3 1 2 3]
 [2 3 4 2 3 4 2 3 4]
 [0 1 2 0 1 2 0 1 2]
 [1 2 3 1 2 3 1 2 3]
 [2 3 4 2 3 4 2 3 4]
 [0 1 2 0 1 2 0 1 2]
 [1 2 3 1 2 3 1 2 3]
 [2 3 4 2 3 4 2 3 4]]


In [35]:
k = np.repeat(np.arange(C,dtype='int32'), field_height * field_width).reshape(-1, 1)
print(k)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [36]:
cols = x_padded[:, k, i, j]
print(cols)

[[[0 0 0 0 1 2 0 4 5]
  [0 0 0 1 2 3 4 5 6]
  [0 0 0 2 3 0 5 6 0]
  [0 1 2 0 4 5 0 7 8]
  [1 2 3 4 5 6 7 8 9]
  [2 3 0 5 6 0 8 9 0]
  [0 4 5 0 7 8 0 0 0]
  [4 5 6 7 8 9 0 0 0]
  [5 6 0 8 9 0 0 0 0]
  [0 0 0 0 1 2 0 4 5]
  [0 0 0 1 2 3 4 5 6]
  [0 0 0 2 3 0 5 6 0]
  [0 1 2 0 4 5 0 7 8]
  [1 2 3 4 5 6 7 8 9]
  [2 3 0 5 6 0 8 9 0]
  [0 4 5 0 7 8 0 0 0]
  [4 5 6 7 8 9 0 0 0]
  [5 6 0 8 9 0 0 0 0]]]


In [37]:
C = x.shape[1]
cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
print(cols)

[[0 0 0 0 1 2 0 4 5]
 [0 0 0 1 2 3 4 5 6]
 [0 0 0 2 3 0 5 6 0]
 [0 1 2 0 4 5 0 7 8]
 [1 2 3 4 5 6 7 8 9]
 [2 3 0 5 6 0 8 9 0]
 [0 4 5 0 7 8 0 0 0]
 [4 5 6 7 8 9 0 0 0]
 [5 6 0 8 9 0 0 0 0]
 [0 0 0 0 1 2 0 4 5]
 [0 0 0 1 2 3 4 5 6]
 [0 0 0 2 3 0 5 6 0]
 [0 1 2 0 4 5 0 7 8]
 [1 2 3 4 5 6 7 8 9]
 [2 3 0 5 6 0 8 9 0]
 [0 4 5 0 7 8 0 0 0]
 [4 5 6 7 8 9 0 0 0]
 [5 6 0 8 9 0 0 0 0]]
